[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-mmlab/mmtracking/blob/master/demo/MMTracking_Tutorial.ipynb)

# **Welcome to MMTracking**

In this tutorial, you will learn to:
+ Install MMTracking.
+ Perform inference with pretrained weights in MMTracking.
+ Train a new MOT model with a toy dataset.
Let's start!

## **Install MMTracking**

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os

root_dir = "/content/drive/My Drive/"

project_folder = "vingroup_training/CV/final_project2/FasterRCNN-DeepSORT"


def create_and_set_working_directory(project_folder):
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')


  os.chdir(root_dir + project_folder)

  print('\nYour working directory was changed to ' + root_dir + project_folder)


create_and_set_working_directory(project_folder)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Your working directory was changed to /content/drive/My Drive/vingroup_training/CV/final_project2/FasterRCNN-DeepSORT


In [2]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
# install pytorch
!pip install torch==1.10.0+cu111 torchvision==0.11.0+cu111 torchaudio==0.10.0 -f https://download.pytorch.org/whl/torch_stable.html

# install MMEngine
!pip install mmengine==0.1.0
# install MMCV
# !pip install mmcv-full
!pip install 'mmcv==2.0.0rc1' -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html

# install MMDetection
!pip install 'mmdet==3.0.0rc2'

# clone the MMTracking repository
!git clone -b 1.x https://github.com/open-mmlab/mmtracking.git
%cd mmtracking

# install MMTracking and its dependencies
!pip install -r requirements/build.txt
!pip install -e .
# used to MOT evaluation
!pip install git+https://github.com/JonathonLuiten/TrackEval.git

Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.10.0+cu111 (from versions: 1.11.0, 1.11.0+cpu, 1.11.0+cu102, 1.11.0+cu113, 1.11.0+cu115, 1.11.0+rocm4.3.1, 1.11.0+rocm4.5.2, 1.12.0, 1.12.0+cpu, 1.12.0+cu102, 1.12.0+cu113, 1.12.0+cu116, 1.12.0+rocm5.0, 1.12.0+rocm5.1.1, 1.12.1, 1.12.1+cpu, 1.12.1+cu102, 1.12.1+cu113, 1.12.1+cu116, 1.12.1+rocm5.0, 1.12.1+rocm5.1.1, 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.0+rocm5.1.1, 1.13.0+rocm5.2, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 1.13.1+rocm5.1.1, 1.13.1+rocm5.2, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.0+rocm5.3, 2.0.0+rocm5.4.2, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2)
ERROR: No matching distribution found for torch==1.10.0+cu111
Look

In [4]:
from mmengine.utils.dl_utils import collect_env
collect_env()

OrderedDict([('sys.platform', 'linux'),
             ('Python', '3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]'),
             ('CUDA available', True),
             ('numpy_random_seed', 2147483648),
             ('GPU 0', 'Tesla T4'),
             ('CUDA_HOME', '/usr/local/cuda'),
             ('NVCC', 'Cuda compilation tools, release 11.8, V11.8.89'),
             ('GCC',
              'x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0'),
             ('PyTorch', '2.0.1+cu118'),
             ('PyTorch compiling details',
              'PyTorch built with:\n  - GCC 9.3\n  - C++ Version: 201703\n  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - LAPACK is enabled (usually provided by MKL)\n  - NNPACK is enabled\n  - CPU capability usage: AVX2\n  - CUDA Runtime 11.8\n  - NVCC archi

In [5]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check MMTracking installation
import mmtrack
print(mmtrack.__version__)

2.0.1+cu118 True
11.8
GCC 11.4
3.0.0rc2
1.0.0rc1


In [6]:
!pip install mmcls==1.0.0rc0

In [13]:
# run mot demo
import mmcv
import mmengine
import tempfile
from mmtrack.apis import inference_mot, init_model
from mmtrack.utils import register_all_modules
from mmtrack.registry import VISUALIZERS

register_all_modules(init_default_scope=True)
mot_config = './configs/mot/deepsort/deepsort_faster-rcnn_r50_fpn_8xb2-4e_mot17halftrain_test-mot17halfval.py'
input_video = './data/demo/demo.webm'
imgs = mmcv.VideoReader(input_video)
# build the model from a config file
mot_model = init_model(mot_config, device='cuda:0')

# build the visualizer. Different name for creating different visualizer instance
mot_model.cfg.visualizer.name = 'mot_visualizer'

visualizer = VISUALIZERS.build(mot_model.cfg.visualizer)
visualizer.dataset_meta = mot_model.dataset_meta

prog_bar = mmengine.ProgressBar(len(imgs))
out_dir = tempfile.TemporaryDirectory()
out_path = out_dir.name

# test and show/save the images
for i, img in enumerate(imgs):
    result = inference_mot(mot_model, img, frame_id=i)
    visualizer.add_datasample(
            'mot',
            img[..., ::-1],
            data_sample=result,
            show=False,
            out_file=f'{out_path}/{i:06d}.jpg',
            wait_time=float(1 / int(imgs.fps)),
            step=i)
    prog_bar.update()

output = './demo/mot.mp4'
print(f'\n making the output video at {output} with a FPS of {imgs.fps}')
mmcv.frames2video(out_path, output, fps=imgs.fps, fourcc='mp4v')
out_dir.cleanup()

09/26 03:37:04 - mmengine - INFO - load model from: ./tutorial_exps/reid/epoch_5.pth
09/26 03:37:04 - mmengine - INFO - local loads checkpoint from path: ./tutorial_exps/reid/epoch_5.pth
09/26 03:37:05 - mmengine - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.fcs.0.fc.weight, head.fcs.0.fc.bias, head.fcs.0.bn.weight, head.fcs.0.bn.bias, head.fcs.0.bn.running_mean, head.fcs.0.bn.running_var, head.fc_out.weight, head.fc_out.bias

missing keys in source state_dict: neck.lateral_convs.0.conv.weight, neck.lateral_convs.0.conv.bias, neck.lateral_convs.1.conv.weight, neck.lateral_convs.1.conv.bias, neck.lateral_convs.2.conv.weight, neck.lateral_convs.2.conv.bias, neck.lateral_convs.3.conv.weight, neck.lateral_convs.3.conv.bias, neck.fpn_convs.0.conv.weight, neck.fpn_convs.0.conv.bias, neck.fpn_convs.1.conv.weight, neck.fpn_convs.1.conv.bias, neck.fpn_convs.2.conv.weight, neck.fpn_convs.2.conv.bias, neck.fpn_convs.3.conv.weight, neck

/content/drive/My Drive/vingroup_training/CV/final_project2/FasterRCNN-DeepSORT/mmtracking/mmtrack/apis/inference.py:90: UserWarning: dataset_meta or class names are missed, use None by default.
  warnings.warn('dataset_meta or class names are missed, '
/usr/local/lib/python3.10/dist-packages/mmengine/visualization/visualizer.py:170: UserWarning: `Visualizer` backend is not initialized because save_dir is None.
  warnings.warn('`Visualizer` backend is not initialized '
/usr/local/lib/python3.10/dist-packages/mmengine/visualization/visualizer.py:707: UserWarning: Warning: The bbox is out of bounds, the drawn bbox may not be in the image
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/mmengine/visualization/visualizer.py:777: UserWarning: Warning: The polygon is out of bounds, the drawn polygon may not be in the image
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/mmengine/visualization/visualizer.py:467: UserWarning: Warning: The text is out of bounds, the drawn text 

[                              ] 21/1050, 1.2 task/s, elapsed: 18s, ETA:   875s

<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
/usr/local/lib/python3.10/dist-packages/matplotlib/artist.py:201: ResourceWarning: unclosed file <_io.BufferedRandom name='/tmp/tmpn6owkk_j'>
  self._callbacks = cbook.CallbackRegistry(signals=["pchanged"])


[>>>>>>>>>>>>>>>>>>>>>>>>>>] 1050/1050, 0.9 task/s, elapsed: 1146s, ETA:     0s
 making the output video at ./demo/mot.mp4 with a FPS of 30.0
[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1050/1050, 32.3 task/s, elapsed: 33s, ETA:     0s


## **Train a MOT model**

### **Prepare dataset**
Convert mot label to reid label

In [ ]:
# convert the dataset to coco format
# !python ./tools/dataset_converters/mot/mot2coco.py -i ./data/MOT20/ -o ./data/MOT20/annotations --split-train --convert-det
# crop pedestrian patches from the original dataset for training reid model. It may take a few minutes.
# !rm -rf ./data/MOT20/reid
!python ./tools/dataset_converters/mot/mot2reid.py -i ./data/MOT20/ -o ./data/MOT20/reid --val-split 0.9 --vis-threshold 0.8

['MOT20-02', 'MOT20-05', 'MOT20-01', 'MOT20-03']
100% 4/4 [14:33<00:00, 218.42s/it]


### **Train a detector for MOT**

In [ ]:
import mmengine
from mmengine.runner import set_random_seed
cfg = mmengine.Config.fromfile('./configs/det/faster-rcnn_r50_fpn_8xb2-4e_mot17halftrain_test-mot17halfval.py')
cfg.data_root = 'data/MOT20/'
cfg.train_dataloader.dataset.data_root = 'data/MOT20/'
cfg.test_dataloader = cfg.test_cfg = cfg.test_evaluator = None
cfg.val_dataloader = cfg.val_cfg = cfg.val_evaluator = None
# different name for creating different visualizer instance
cfg.visualizer.name = 'mot_visualizer'


cfg.work_dir = './tutorial_exps/detector'
cfg.randomness = dict(seed=0, deterministic=False)
cfg.gpu_ids = range(1)
print(f'Config:\n{cfg.pretty_text}')

Config:
dataset_type = 'mmdet.CocoDataset'
data_root = 'data/MOT20/'
train_pipeline = [
    dict(type='LoadImageFromFile', to_float32=True),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='RandomResize',
        scale=(1088, 1088),
        ratio_range=(0.8, 1.2),
        keep_ratio=True,
        clip_object_border=False),
    dict(type='PhotoMetricDistortion'),
    dict(type='RandomCrop', crop_size=(1088, 1088), bbox_clip_border=False),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PackDetInputs')
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', scale=(1088, 1088), keep_ratio=True),
    dict(
        type='PackDetInputs',
        meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape',
                   'scale_factor'))
]
train_dataloader = dict(
    batch_size=2,
    num_workers=2,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=True),
    batch_sampler=dict(type='mmdet.AspectRatioBatchSam

In [ ]:
import os.path as osp

from mmengine.utils import mkdir_or_exist
from mmengine.runner import Runner

mkdir_or_exist(osp.abspath(cfg.work_dir))
runner = Runner.from_cfg(cfg)
runner.train()

09/18 03:26:53 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
    CUDA available: True
    numpy_random_seed: 0
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.0.1+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm

Downloading: "http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth" to /root/.cache/torch/hub/checkpoints/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth
100%|██████████| 160M/160M [00:05<00:00, 31.7MB/s]


09/18 03:31:35 - mmengine - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([2, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([2]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([4, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([4]).
09/18 03:31:35 - mmengine - INFO - Checkpoints will be saved to /content/drive/My Drive/final_project2/mmtracking/tutorial_exps/detector by HardDiskBackend.
09/18 03:33:13 - mmengine - INFO - Epoch(train) [1][50/2234]  lr: 1.0000e-02  eta: 12:08:

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Custom TB Handler failed, unregistering
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-f71e0f142884>", line 8, in <cell line: 8>
    runner.train()
  File "/usr/local/lib/python3.10/dist-packages/mmengine/runner/runner.py", line 1631, in train
    model = self.train_loop.run()  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/mmengine/runner/loops.py", line 88, in run
    self.run_epoch()
  File "/usr/local/lib/python3.10/dist-packages/mmengine/runner/loops.py", line 103, in run_epoch
    for idx, data_batch in enumerate(self.dataloader):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 633, in __next__
    data = self._next_data()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1345, in _next_data
    return self._process_data(data)
  File "/usr/local/lib/python3.10/

### **Train a ReID model for MOT**

In [7]:
!pip install yapf==0.40.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: yapf
    Found existing installation: yapf 0.40.2
    Uninstalling yapf-0.40.2:
      Successfully uninstalled yapf-0.40.2


In [ ]:
import mmengine

cfg = mmengine.Config.fromfile('./configs/reid/reid_r50_8xb32-6e_mot17train80_test-mot17val20.py')
cfg.train_dataloader.dataset.data_root = 'data/MOT20/'
cfg.train_dataloader.dataset.ann_file = 'newreid/meta/train_9.txt'
cfg.test_dataloader = cfg.test_cfg = cfg.test_evaluator = None
cfg.val_dataloader = cfg.val_cfg = cfg.val_evaluator = None
cfg.visualizer.name = 'mot_reid_visualizer'


# learning policy
cfg.param_scheduler = [
    dict(
        type='LinearLR',
        start_factor=1.0 / 200,
        by_epoch=False,
        begin=0,
        end=200),
    dict(
        type='MultiStepLR',
        begin=0,
        end=2,
        by_epoch=True,
        milestones=[1],
        gamma=0.1)
]
cfg.train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=5, val_begin=3)

cfg.work_dir = './tutorial_exps/reid'
cfg.randomness = dict(seed=0, deterministic=False)
cfg.gpu_ids = range(1)
print(f'Config:\n{cfg.pretty_text}')

In [ ]:
import os.path as osp

from mmengine.utils import mkdir_or_exist
from mmengine.runner import Runner

mkdir_or_exist(osp.abspath(cfg.work_dir))
runner = Runner.from_cfg(cfg)
runner.train()

### **Test the FasterRCNN - DeepSORT model**

In [ ]:
!pip install yapf==0.40.1

test on MOT17

In [ ]:
import mmengine

cfg = mmengine.Config.fromfile('./configs/mot/deepsort/deepsort_faster-rcnn_r50_fpn_8xb2-4e_mot17halftrain_test-mot17halfval.py')
cfg.test_dataloader.dataset.data_root = 'data/MOT17_tiny/'
cfg.test_dataloader.dataset.test_mode = True
cfg.train_dataloader = cfg.train_cfg = None
cfg.val_dataloader = cfg.val_cfg = cfg.val_evaluator = None
cfg.visualizer.name = 'deepsort_visualizer'
cfg.model.detector.init_cfg.checkpoint = './tutorial_exps/detector/epoch_5.pth'
cfg.model.reid.init_cfg.checkpoint = './tutorial_exps/reid/epoch_5.pth'

cfg.work_dir = './tutorial_exps'
cfg.randomness = dict(seed=0, deterministic=False)
cfg.gpu_ids = range(1)
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    data_preprocessor=dict(
        type='TrackDataPreprocessor',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        bgr_to_rgb=True,
        rgb_to_bgr=False,
        pad_size_divisor=32),
    detector=dict(
        type='FasterRCNN',
        _scope_='mmdet',
        backbone=dict(
            type='ResNet',
            depth=50,
            num_stages=4,
            out_indices=(0, 1, 2, 3),
            frozen_stages=1,
            norm_cfg=dict(type='BN', requires_grad=True),
            norm_eval=True,
            style='pytorch',
            init_cfg=dict(
                type='Pretrained', checkpoint='torchvision://resnet50')),
        neck=dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            num_outs=5),
        rpn_head=dict(
            type='RPNHead',
            in_channels=256,
            feat_channels=256,
            anchor_generator=dict(
          

In [ ]:
from mmengine.model import is_model_wrapper
from mmengine.runner import Runner

runner = Runner.from_cfg(cfg)

if is_model_wrapper(runner.model):
    runner.model.module.init_weights()
else:
    runner.model.init_weights()

runner.test()

09/25 05:00:32 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
    CUDA available: True
    numpy_random_seed: 0
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.0.1+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm

{'motchallenge-metric/HOTA': 0.5884722541229944,
 'motchallenge-metric/AssA': 0.610290244906357,
 'motchallenge-metric/DetA': 0.5787586847661333,
 'motchallenge-metric/MOTA': 0.6756709143226262,
 'motchallenge-metric/MOTP': 0.8131037010815175,
 'motchallenge-metric/IDSW': 438.0,
 'motchallenge-metric/TP': 25694.0,
 'motchallenge-metric/FP': 2244.0,
 'motchallenge-metric/FN': 8364.0,
 'motchallenge-metric/Frag': 566.0,
 'motchallenge-metric/MT': 62.0,
 'motchallenge-metric/ML': 12.0,
 'motchallenge-metric/IDF1': 0.709239305761662,
 'motchallenge-metric/IDTP': 21985.0,
 'motchallenge-metric/IDFN': 12073.0,
 'motchallenge-metric/IDFP': 5953.0,
 'motchallenge-metric/IDP': 0.7869210394444842,
 'motchallenge-metric/IDR': 0.645516471900875}

<Figure size 640x480 with 0 Axes>

test on MOT20

In [8]:
import mmengine

cfg = mmengine.Config.fromfile('./configs/mot/deepsort/deepsort_faster-rcnn_r50_fpn_8xb2-4e_mot17halftrain_test-mot17halfval.py')
cfg.test_dataloader.dataset.data_root = 'data/MOT20/'
cfg.test_dataloader.dataset.test_mode = True
cfg.train_dataloader = cfg.train_cfg = None
cfg.val_dataloader = cfg.val_cfg = cfg.val_evaluator = None
cfg.visualizer.name = 'deepsort_visualizer'
cfg.model.detector.init_cfg.checkpoint = './tutorial_exps/detector/epoch_5.pth'
cfg.model.reid.init_cfg.checkpoint = './tutorial_exps/reid/epoch_5.pth'

cfg.work_dir = './tutorial_exps'
cfg.randomness = dict(seed=0, deterministic=False)
cfg.gpu_ids = range(1)
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    data_preprocessor=dict(
        type='TrackDataPreprocessor',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        bgr_to_rgb=True,
        rgb_to_bgr=False,
        pad_size_divisor=32),
    detector=dict(
        type='FasterRCNN',
        _scope_='mmdet',
        backbone=dict(
            type='ResNet',
            depth=50,
            num_stages=4,
            out_indices=(0, 1, 2, 3),
            frozen_stages=1,
            norm_cfg=dict(type='BN', requires_grad=True),
            norm_eval=True,
            style='pytorch',
            init_cfg=dict(
                type='Pretrained', checkpoint='torchvision://resnet50')),
        neck=dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            num_outs=5),
        rpn_head=dict(
            type='RPNHead',
            in_channels=256,
            feat_channels=256,
            anchor_generator=dict(
          

In [9]:
from mmengine.model import is_model_wrapper
from mmengine.runner import Runner

runner = Runner.from_cfg(cfg)

if is_model_wrapper(runner.model):
    runner.model.module.init_weights()
else:
    runner.model.init_weights()

runner.test()

09/26 02:53:31 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
    CUDA available: True
    numpy_random_seed: 0
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.0.1+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm

{'motchallenge-metric/HOTA': 0.2761230873318859,
 'motchallenge-metric/AssA': 0.2296492927689919,
 'motchallenge-metric/DetA': 0.3360262016183487,
 'motchallenge-metric/MOTA': 0.4163267694838711,
 'motchallenge-metric/MOTP': 0.7154844231673503,
 'motchallenge-metric/IDSW': 9625.0,
 'motchallenge-metric/TP': 292991.0,
 'motchallenge-metric/FP': 27268.0,
 'motchallenge-metric/FN': 322146.0,
 'motchallenge-metric/Frag': 21436.0,
 'motchallenge-metric/MT': 149.0,
 'motchallenge-metric/ML': 406.0,
 'motchallenge-metric/IDF1': 0.3655991686943284,
 'motchallenge-metric/IDTP': 170990.0,
 'motchallenge-metric/IDFN': 444147.0,
 'motchallenge-metric/IDFP': 149269.0,
 'motchallenge-metric/IDP': 0.5339116152863775,
 'motchallenge-metric/IDR': 0.2779705984195391}

<Figure size 640x480 with 0 Axes>